# Qutrit

For more details about the formalism and what we are computing, check the accompanying notebook "open_qubit.ipynb".

We choose the measuring basis to be $M = \{|m(\pm)\rangle = |\pm1, S_z\rangle, |m(0)\rangle = |0, S_z\rangle \}$. The Hamiltonian is $S_x$, where $S$'s are spin-1 matrices.

In [1]:
import numpy as np
from scipy.linalg import expm

In [8]:
def time_evolve(state, t, w):
    
    """Takes state and evolves it unitarily
       with Hamiltonian H for time t"""
    
    H = np.array([[0, 1., 0], [1., 0, 1.], [0, 1., 0]])/np.sqrt(2.)
    
    return expm(-1j*H*w*t)@state



def m(sign):
    
    """Returns state in the Sz eigenbasis"""
    
    if sign == '+':
        
        return np.array([1, 0, 0])
    
    elif sign == '0':
        
        return np.array([0, 1, 0])
    
    elif sign == '-':
        
        return np.array([0, 0, 1])
    

def probs(state):
    
    """Takes a state and returns the probabilities 
       of measuring state m(+), m(0) and m(-)"""
    
    p = np.zeros(3)
    
    mp = m('+')
    m0 = m('0')
    mm = m('-')
    
    p[0] = abs(np.dot(mp.conj().T, state))**2
    p[1] = abs(np.dot(m0.conj().T, state))**2
    p[2] = abs(np.dot(mm.conj().T, state))**2
    
    return p

def measure(state):
    
    """Measures input state and outputs measured state and its label"""
    
    p = probs(state)
    choice = np.random.choice(3, p=p.tolist())
    
    if choice == 0:
        # This is m+
        return [m('+'), '+']
    elif choice == 1:
        # This is m0
        return [m('0'), '0']
    elif choice == 2:
        # This is m-
        return [m('-'), '-']

    
def T(in_state, target_state, tau, omega, ntimes):
    
    """Returns mean time T(in,out) assuming exponential pdf with mean tau"""
    
    T_list = []
    
    for n in range(ntimes):
        
        t_list = []

        # Sample measuring time from pdf
        t = np.random.exponential(tau)
        t_list.append(t)

        # Prepare initial state
        psi0 = m(in_state)
        
        # Evolve for time t
        psi = time_evolve(psi0, t, omega)

        # Measure state at time t
        psi, lf = measure(psi)
        

        # Iterate until you find target state
        while lf != target_state:
            
            t = np.random.exponential(tau)
            t_list.append(t)

            psi = time_evolve(psi, t, omega)

            psi, lf = measure(psi)

        T_list.append(sum(t_list))

    Ts = np.array(T_list)
    Ts = np.mean(Ts)
    
    return Ts

In [9]:
tau = 1
omega = 1
ntimes = 1000


Tpp = T('+','+', tau, omega, ntimes)
T00 = T('0','0', tau, omega, ntimes)
Tmm = T('-','-', tau, omega, ntimes)
Tpm = T('+','-', tau, omega, ntimes)
Tmp = T('-','+', tau, omega, ntimes)
Tp0 = T('+','0', tau, omega, ntimes)
Tm0 = T('-','0', tau, omega, ntimes)
T0p = T('0','+', tau, omega, ntimes)
T0m = T('0','-', tau, omega, ntimes)

print('---- T(in,out) ----')
print('T++:', round(Tpp, 2), 'Theory:', 3)
print('T00:', round(T00, 2), 'Theory:', 3)
print('T--:', round(Tmm, 2), 'Theory:', 3)
print('T+-:', round(Tpm, 2), 'Theory:', 6)
print('T-+:', round(Tmp, 2), 'Theory:', 6)
print('T+0:', round(Tp0, 2), 'Theory:', 5)
print('T-0:', round(Tm0, 2), 'Theory:', 5)
print('T0+:', round(T0p, 2), 'Theory:', 5.5)
print('T0-:', round(T0m, 2), 'Theory:', 5.5)

---- T(in,out) ----
T++: 2.99 Theory: 3
T00: 3.02 Theory: 3
T--: 2.99 Theory: 3
T+-: 5.95 Theory: 6
T-+: 6.18 Theory: 6
T+0: 4.98 Theory: 5
T-0: 5.13 Theory: 5
T0+: 5.68 Theory: 5.5
T0-: 5.56 Theory: 5.5
